# 23) Collective communication

Last time:
- Blocking and non-blocking point-to-point communications

Today:
1. MPI Collective Communication  
2. Minimum Spanning Trees  


## 1. MPI Collective Communication

:::{tip}
Resources for the lecture:
1. Article: Chan et al., [_Collective communication: theory, practice, and experience_](https://csu-sdsu.primo.exlibrisgroup.com/permalink/01CALS_SDL/10r4g1c/cdi_crossref_primary_10_1002_cpe_1206)
2. Lecture Notes from the University of Texas at Austin: Robert van de Geijn (RVDG) [_Collective Communication: Theory and Practice_](https://www.cs.utexas.edu/~rvdg/tmp/CollectiveCommunication.pdf)
:::

In the previous lecture we considered point-to-point communication, that is communication between two MPI ranks. In this lecture we will consider communication that involves _all_ the ranks in a communicator.

### Reductions

Recall that [reductions](https://en.wikipedia.org/wiki/Reduction_operator) are a case of collective communication operations. 

> An operator is a reduction operator if:
> 1. It can reduce an array to a single scalar value.
> 2. The final result should be obtainable from the results of the partial tasks that were created.

These two requirements are satisfied for **commutative** and **associative** operators that are applied to _all_ array elements.

#### Example

Suppose we have an array $x = [1,2,3,4,5,6,7,8]$. The sum of this array can be computed _serially_ by sequentially reducing the array into a single sum using the `+` operator. 

Starting the summation from the beginning of the array yields:

$$
(((((((1 + 2) + 3) + 4) + 5) + 6) + 7) + 8)
$$

Since `+` is both _commutative_ and _associative_, it is a reduction operator. Therefore this reduction can be performed in parallel using several processes/cores, where each process/core computes the sum of a subset of the array, and the reduction operator merges the results. 

Using a [binary tree](https://en.wikipedia.org/wiki/Binary_tree) reduction would allow $4$ processes/cores to compute 


$$
\underbrace{ \underbrace{ \underbrace{(1 + 2)}_{p_0} + \underbrace{(3 + 4)}_{p_1}}_{p_0} + \underbrace{ \underbrace{(5 + 6)}_{p_1} + \underbrace{(7 + 8)}_{p_3}}_{p_1} }_{p_0}
$$

So a total of $4$ cores can be used to compute the sum in $\log_n \equiv \log_2 8 = 3$ steps instead of the $7$ steps required for the serial version. 

Of course the result is the same, but only because of the associativity of the reduction operator. The commutativity of the reduction operator would be important if there were a master core distributing work to several processors, since then the results could arrive back to the master processor in any order. The property of commutativity guarantees that the result will be the same. 




- From Fig. 1 in Chan et al.

![Figure 1 of Chang et al, Collective communication: theory, practice, and experience article](../img/MPI_collective_comm_Chang_et_al_fig1.png)

Types of possible collective communication:

- Broadcast: one rank sends data to all the other ranks (RVDG: 93, `MPI_Bcast`)
- Reduce(-to-one): Combine (e.g., `sum`, `max`/`min`, etc.) information from all ranks to one rank (RVDG: 94, `MPI_Reduce`)
- Scatter: One rank send data to all other ranks (RVDG: 96, `MPI_Scatter`)
- Gather: All ranks send data to one rank (RVDG: 97, `MPI_Gather`)
- Allgather: All ranks sends information to all ranks (RVDG: 99, `MPI_Allgather`)
- Reduce-scatter: Reduce and scatter out the reduced result (RVDG: 101, `MPI_Reduce_scatter`)
- Allreduce: All ranks combine information from all ranks and the result is available to all ranks (RVDG: 102, `MPI_Allreduce`)

Note that there are pairs of reciprocal/dual operations:

- Broadcast/Reduce(-to-one) (RVDG: 95)
- Scatter/Gather (RVDG: 98)
- Allgather/Reduce-scatter (RVDG: 101)

Allreduce is the only operationthat does not have a dual (or it can be viewed as its own dual).

Two broad classes of collective operations (Chan et al., 1752):

> - _Data redistribution operations:_ Broadcast, scatter, gather, and allgather. These operations move data between processors.
> - _Data consolidation operations:_ Reduce(-to-one), reduce–scatter, and allreduce. These operations consolidate contributions from different processors by applying a reduction operation. We will only consider reduction operations that are both commutative and associative.


## 2. [Minimum Spanning Trees (MST)](https://en.wikipedia.org/wiki/Parallel_algorithms_for_minimum_spanning_trees)

### 2.1 Broadcast

We want to perform a broadcast operation, i.e., we want to send a message from a root rank to all ranks.

#### Naive Algorithm

A naive broadcast just has the root send the message to each rank.

You can find the following code at [`julia_codes/module6-3/naivebcast.jl`](https://github.com/sdsu-comp605/spring25/tree/main/julia_codes/julia_codes/module6-3/naivebcast.jl).

```{literalinclude} ../julia_codes/module6-3/naivebcast.jl
:language: julia
:linenos: true
```

And the following testing code at [`julia_codes/module6-3/naivebcast_test.jl`](https://github.com/sdsu-comp605/spring25/tree/main/julia_codes/julia_codes/module6-3/naivebcast_test.jl).

```{literalinclude} ../julia_codes/module6-3/naivebcast_test.jl
:language: julia
:linenos: true
```

#### Minimum Spanning Tree Algorithm
See Chan et al. Fig. 3(a) & Fig 4(a); RVDG pages 108-171.

You can find the following code at [`julia_codes/module6-3/mstbcast.jl`](https://github.com/sdsu-comp605/spring25/tree/main/julia_codes/julia_codes/module6-3/mstbcast.jl).

```{literalinclude} ../julia_codes/module6-3/mstbcast.jl
:language: julia
:linenos: true
```

And the following testing code at [`julia_codes/module6-3/mstbcast_test.jl`](https://github.com/sdsu-comp605/spring25/tree/main/julia_codes/julia_codes/module6-3/mstbcast_test.jl).

```{literalinclude} ../julia_codes/module6-3/mstbcast_test.jl
:language: julia
:linenos: true
```


#### Let's compare them

You can find the driver for the comparison code at [`julia_codes/module6-3/bcast_compare.jl`](https://github.com/sdsu-comp605/spring25/tree/main/julia_codes/julia_codes/module6-3/bcast_compare.jl).

```{literalinclude} ../julia_codes/module6-3/bcast_compare.jl
:language: julia
:linenos: true
```

### Minimum Spanning Tree Reduce?

For a **Minimum Spanning Tree (MST) Reduce** see See Chan et al. Fig. 3(b) & Fig. 4(b); RVDG pages 172-184.

- **Note**: It's possible to write the `mstbcast` (and a corresponding MST reduce version) without recursion (although we won't see it in this class) and doing it with blocking communication is easier than non-blocking...

